In [1]:
# download libraries and packages
!pip install gensim
!pip install python-docx

In [3]:
# import libraries and packages
import pandas as pd
import numpy as np
import gensim
from docx import Document

ImportError: cannot import name 'triu' from 'scipy.linalg' (/opt/anaconda3/lib/python3.12/site-packages/scipy/linalg/__init__.py)

In [10]:
# import data (text file)
file_name = "Sample Data for Evaluation.docx"
doc = Document(file_name)

# extract text from doc object
text = "\n".join([paragraph.text for paragraph in doc.paragraphs])

In [15]:
# remove unwanted text #
import re

# remove emails
text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)

# remove URLs
text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

# normalize white space
text = re.sub(r'\s+', ' ', text).strip()

In [17]:
# text tokenization #
from gensim.utils import simple_preprocess

# tokenize the text
tokens = simple_preprocess(text)

In [23]:
# stop words removal #
from nltk.corpus import stopwords
import nltk

# load built-in stop words
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# remove stop words
tokens = [token for token in tokens if token not in stop_words]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jenni\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [30]:
# lemmatization #
!python -m spacy download en_core_web_md
import spacy

# load spacey model
nlp = spacy.load('en_core_web_md')

# lemmatize using spacey
text = ' '.join(tokens)
doc = nlp(text)
tokens = [token.lemma_ for token in doc]

     ---------------------------------------- 0.0/42.8 MB ? eta -:--:--
     --------------------------------------- 0.0/42.8 MB 330.3 kB/s eta 0:02:10
     --------------------------------------- 0.1/42.8 MB 787.7 kB/s eta 0:00:55
     ---------------------------------------- 0.3/42.8 MB 2.6 MB/s eta 0:00:17
     - -------------------------------------- 1.1/42.8 MB 6.5 MB/s eta 0:00:07
     - -------------------------------------- 1.7/42.8 MB 7.7 MB/s eta 0:00:06
     -- ------------------------------------- 2.5/42.8 MB 9.5 MB/s eta 0:00:05
     -- ------------------------------------- 2.7/42.8 MB 9.5 MB/s eta 0:00:05
     -- ------------------------------------- 3.0/42.8 MB 8.4 MB/s eta 0:00:05
     --- ------------------------------------ 3.8/42.8 MB 9.4 MB/s eta 0:00:05
     --- ------------------------------------ 4.1/42.8 MB 9.4 MB/s eta 0:00:05
     --- ------------------------------------ 4.1/42.8 MB 9.4 MB/s eta 0:00:05
     ---- ----------------------------------- 5.0/42.8 MB

In [31]:
print(text)

unacceptable risk scenario social scoring system description government implements ai system assigns scores citizens based social behavior financial status online activities risk could lead discrimination loss privacy social exclusion making unacceptable risk eu ai act high risk scenario ai healthcare description hospital uses ai system diagnose diseases recommend treatments based patient data risk incorrect diagnoses treatment recommendations could serious health consequences therefore ai system subject strict regulations oversight scenario autonomous vehicles description company deploys self driving cars public roads risk malfunctions errors ai system could lead accidents posing significant safety risks systems must comply rigorous safety standards limited risk scenario customer service chatbots description commerce website uses ai powered chatbots handle customer inquiries support risk risk lower transparency required inform users interacting ai ensuring ethical use scenario recomme

In [ ]:
def parse_tokens(text_tokens):
    structured_data = []
    current_dict = {}
    i = 0
    risk_levels = ["unacceptable", "high", "limited", "minimal"]

    while i < len(text_tokens):
        if text_tokens[i] in risk_levels and text_tokens[i+1] == "risk":
            # If a dictionary has been started, finalize it and start a new one
            if current_dict and current_dict.get("scenario"):
                structured_data.append(current_dict)
            # Initialize a new dictionary
            current_dict = {
                "risk_level": f"{text_tokens[i]} risk".capitalize(),
                "scenario": "",
                "description": [],
                "risk": []
            }
            i += 2
        elif text_tokens[i] == "scenario":
            i += 1
            scenario = []
            while i < len(text_tokens) and text_tokens[i] != "description":
                scenario.append(text_tokens[i])
                i += 1
            current_dict["scenario"] = " ".join(scenario).capitalize()
        elif text_tokens[i] == "description":
            i += 1
            description = []
            while i < len(text_tokens) and text_tokens[i] != "risk" and text_tokens[i] != "scenario":
                description.append(text_tokens[i])
                i += 1
            # Assign description words to a list
            current_dict["description"] = description
        elif text_tokens[i] == "risk":
            i += 1
            risk = []
            while i < len(text_tokens) and text_tokens[i] not in risk_levels and text_tokens[i] != "scenario":
                risk.append(text_tokens[i])
                i += 1
            # Append risk words to the list
            if current_dict.get("risk"):
                current_dict["risk"].extend(risk)
            else:
                current_dict["risk"] = risk
        else:
            i += 1
    
    # Final check to append the last dictionary if it's fully populated
    if current_dict and current_dict.get("scenario"):
        structured_data.append(current_dict)
    
    return structured_data

# Sample text
text = """unacceptable risk scenario social scoring system description government implements ai system assigns scores citizens based social behavior financial status online activities risk could lead discrimination loss privacy social exclusion making unacceptable risk eu ai act high risk scenario ai healthcare description hospital uses ai system diagnose diseases recommend treatments based patient data risk incorrect diagnoses treatment recommendations could serious health consequences therefore ai system subject strict regulations oversight scenario autonomous vehicles description company deploys self driving cars public roads risk malfunctions errors ai system could lead accidents posing significant safety risks systems must comply rigorous safety standards limited risk scenario customer service chatbots description commerce website uses ai powered chatbots handle customer inquiries support risk risk lower transparency required inform users interacting ai ensuring ethical use scenario recommendation systems description streaming service uses ai suggest movies shows based user preferences risk main concern ensuring users aware data used manageable transparency obligations minimal risk scenario spam filters description email service provider uses ai filter spam phishing emails risk risk minimal systems designed enhance user experience without significant negative impacts scenario ai powered games description video game developer uses ai create adaptive intelligent non player characters npcs enhance gameplay risk systems pose minimal risk primarily entertainment purposes"""

# Tokenize the text
text_tokens = text.split()

# Parse tokens into structured data
structured_data = parse_tokens(text_tokens)

# Display the structured data
for item in structured_data:
    print(item)